##Load the datset

In [38]:
import pandas as pd
data= pd.read_csv('/content/Expresso_churn_dataset.csv')

#Load the dataset

In [39]:
pip install --upgrade pandas-profiling

In [40]:
pip install numba==0.56.4


  Using cached numba-0.56.4.tar.gz (2.4 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [41]:
 import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.impute import SimpleImputer

import streamlit as st
import uuid

In [42]:
!pip install streamlit

#Data exploration

In [43]:
data.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,NO,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,NO,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,NO,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,NO,11,Mixt 250F=Unlimited_call24H,2.0,0


In [44]:
data.tail()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
2154043,ffffe85215ddc71a84f95af0afb0deeea90e6967,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,6,NaN,NaN,0
2154044,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15,THIES,K > 24 month,6100.0,15.0,5800.0,1933.0,15.0,621.0,26.0,40.0,40.0,NaN,NaN,NO,55,"Data: 200 F=100MB,24H",9.0,0
2154045,fffff172fda1b4bb38a95385951908bb92379809,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,1,NaN,NaN,1
2154046,fffff5911296937a37f09a37a549da2e0dad6dbb,THIES,K > 24 month,10000.0,11.0,7120.0,2373.0,13.0,NaN,0.0,140.0,13.0,NaN,NaN,NO,28,All-net 500F=2000F;5d,12.0,0
2154047,fffff6dbff1508ea2bfe814e5ab2729ce6b788c2,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NO,62,NaN,NaN,1


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2154048 entries, 0 to 2154047
Data columns (total 19 columns):
 #   Column          Dtype  
---  ------          -----  
 0   user_id         object 
 1   REGION          object 
 2   TENURE          object 
 3   MONTANT         float64
 4   FREQUENCE_RECH  float64
 5   REVENUE         float64
 6   ARPU_SEGMENT    float64
 7   FREQUENCE       float64
 8   DATA_VOLUME     float64
 9   ON_NET          float64
 10  ORANGE          float64
 11  TIGO            float64
 12  ZONE1           float64
 13  ZONE2           float64
 14  MRG             object 
 15  REGULARITY      int64  
 16  TOP_PACK        object 
 17  FREQ_TOP_PACK   float64
 18  CHURN           int64  
dtypes: float64(12), int64(2), object(5)
memory usage: 312.2+ MB


In [46]:
data.describe(include='all')

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
count,2154048,1304749,2154048,1.397309e+06,1.397309e+06,1.428000e+06,1.428000e+06,1.428000e+06,1.093615e+06,1.367373e+06,1.258800e+06,864032.000000,169721.000000,136824.000000,2154048,2.154048e+06,1251454,1.251454e+06,2.154048e+06
unique,2154048,14,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,140,NaN,NaN
top,ffff54079880e00af686f7d6403dd506c9560430,DAKAR,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NaN,All-net 500F=2000F;5d,NaN,NaN
freq,1,513271,2043201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2154048,NaN,317802,NaN,NaN
mean,NaN,NaN,NaN,5.532117e+03,1.152912e+01,5.510810e+03,1.836943e+03,1.397814e+01,3.366450e+03,2.776891e+02,9.541871e+01,23.109253,8.170132,7.553309,NaN,2.804251e+01,NaN,9.272461e+00,1.875474e-01
std,NaN,NaN,NaN,7.111339e+03,1.327407e+01,7.187113e+03,2.395700e+03,1.469403e+01,1.330446e+04,8.726889e+02,2.049873e+02,63.578086,41.169511,33.487234,NaN,2.228686e+01,NaN,1.228044e+01,3.903504e-01
min,NaN,NaN,NaN,1.000000e+01,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,NaN,1.000000e+00,NaN,1.000000e+00,0.000000e+00
25%,NaN,NaN,NaN,1.000000e+03,2.000000e+00,1.000000e+03,3.330000e+02,3.000000e+00,0.000000e+00,5.000000e+00,7.000000e+00,2.000000,0.000000,0.000000,NaN,6.000000e+00,NaN,2.000000e+00,0.000000e+00
50%,NaN,NaN,NaN,3.000000e+03,7.000000e+00,3.000000e+03,1.000000e+03,9.000000e+00,2.570000e+02,2.700000e+01,2.900000e+01,6.000000,1.000000,2.000000,NaN,2.400000e+01,NaN,5.000000e+00,0.000000e+00
75%,NaN,NaN,NaN,7.350000e+03,1.600000e+01,7.368000e+03,2.456000e+03,2.000000e+01,2.895000e+03,1.560000e+02,9.900000e+01,20.000000,3.000000,5.000000,NaN,5.100000e+01,NaN,1.200000e+01,0.000000e+00


#checking for nay missing values

In [47]:
data.isnull().sum()

,0
user_id,0
REGION,849299
TENURE,0
MONTANT,756739
FREQUENCE_RECH,756739
REVENUE,726048
ARPU_SEGMENT,726048
FREQUENCE,726048
DATA_VOLUME,1060433
ON_NET,786675


#Understand how much is missing in percentage

In [48]:
missing_percent = (data.isnull().sum() / len(data)) * 100
print(missing_percent.sort_values(ascending=False))


ZONE2             93.648052
ZONE1             92.120835
TIGO              59.887988
DATA_VOLUME       49.229776
TOP_PACK          41.902223
FREQ_TOP_PACK     41.902223
ORANGE            41.561191
REGION            39.428044
ON_NET            36.520774
MONTANT           35.131018
FREQUENCE_RECH    35.131018
FREQUENCE         33.706213
ARPU_SEGMENT      33.706213
REVENUE           33.706213
user_id            0.000000
TENURE             0.000000
MRG                0.000000
REGULARITY         0.000000
CHURN              0.000000
dtype: float64


#Handling the cleaning and the missing values

In [49]:
# DROP COLUMNS WITH > 50% MISSING DATA
data.drop(['ZONE1', 'ZONE2', 'TIGO'], axis=1, inplace=True,errors="ignore")

# IMPUTE CATEGORICAL COLUMNS WITH MODE
data['REGION'].fillna(data['REGION'].mode()[0], inplace=True)
data['TOP_PACK'].fillna(data['TOP_PACK'].mode()[0], inplace=True)

# IMPUTE NUMERICAL COLUMNS WITH MEDIAN
num_cols = ['FREQUENCE_RECH', 'MONTANT', 'REVENUE', 'ARPU_SEGMENT',
            'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'FREQ_TOP_PACK']

for col in num_cols:
    data[col].fillna(data[col].median(), inplace=True)

# FINAL CHECK
print("✅ Missing values after cleaning:\n")
print(data.isnull().sum())


/tmp/ipython-input-49-2263257450.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['REGION'].fillna(data['REGION'].mode()[0], inplace=True)
/tmp/ipython-input-49-2263257450.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inp

✅ Missing values after cleaning:

user_id           0
REGION            0
TENURE            0
MONTANT           0
FREQUENCE_RECH    0
REVENUE           0
ARPU_SEGMENT      0
FREQUENCE         0
DATA_VOLUME       0
ON_NET            0
ORANGE            0
MRG               0
REGULARITY        0
TOP_PACK          0
FREQ_TOP_PACK     0
CHURN             0
dtype: int64


In [50]:
data.columns

Index(['user_id', 'REGION', 'TENURE', 'MONTANT', 'FREQUENCE_RECH', 'REVENUE',
       'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'MRG',
       'REGULARITY', 'TOP_PACK', 'FREQ_TOP_PACK', 'CHURN'],
      dtype='object')

#Handle duplicates

In [51]:
#checking the duplicates
data.duplicated().sum()

np.int64(0)

#Handling encoding

In [52]:
#know the data to convert to columns
data.dtypes

,0
user_id,object
REGION,object
TENURE,object
MONTANT,float64
FREQUENCE_RECH,float64
REVENUE,float64
ARPU_SEGMENT,float64
FREQUENCE,float64
DATA_VOLUME,float64
ON_NET,float64


In [53]:
# Step 1: Import pandas (if not already)
import pandas as pd

# Step 2: Load your dataset (assuming it's in a DataFrame called df)

# Step 3: Apply one-hot encoding to categorical columns
categorical_cols = ['REGION', 'MRG', 'TOP_PACK']

# Create dummy variables
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

# Step 4: Confirm changes
print(data_encoded.head())
print(data_encoded.columns)


                                    user_id         TENURE  MONTANT  \
0  00000bfd7d50f01092811bc0c8d7b0d6fe7c3596   K > 24 month   4250.0   
1  00000cb4a5d760de88fecb38e2f71b7bec52e834  I 18-21 month   3000.0   
2  00001654a9d9f96303d9969d0a4a851714a4bb57   K > 24 month   3600.0   
3  00001dd6fa45f7ba044bd5d84937be464ce78ac2   K > 24 month  13500.0   
4  000028d9e13a595abe061f9b58f3d76ab907850f   K > 24 month   1000.0   

   FREQUENCE_RECH  REVENUE  ARPU_SEGMENT  FREQUENCE  DATA_VOLUME  ON_NET  \
0            15.0   4251.0        1417.0       17.0          4.0   388.0   
1             7.0   3000.0        1000.0        9.0        257.0    27.0   
2             2.0   1020.0         340.0        2.0        257.0    90.0   
3            15.0  13502.0        4501.0       18.0      43804.0    41.0   
4             1.0    985.0         328.0        1.0        257.0    39.0   

   ORANGE  ...  TOP_PACK_WIFI_ Family _4MBPS  TOP_PACK_WIFI_Family_2MBPS  \
0    46.0  ...                         F

#Machine learning

In [54]:
pip install xgboost


In [55]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


#Define the the term

In [57]:
# Drop user_id and target column to get X
X = data.drop(columns=['user_id', 'CHURN'])

# Set the target variable
y = data['CHURN']


#Split the data

In [58]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [59]:
model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss'
)

model.fit(X_train, y_train)


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:REGION: object, TENURE: object, MRG: object, TOP_PACK: object

In [60]:
# Apply One-Hot Encoding
data_encoded = pd.get_dummies(data, columns=['REGION', 'TENURE', 'MRG', 'TOP_PACK'])

# Now separate features and target again
X = data_encoded.drop(columns=['user_id', 'CHURN'])
y = data_encoded['CHURN']

# Train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [61]:
import xgboost as xgb

model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss'
)

model.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:19:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#Drop user id